In [ ]:
from google.colab import drive
drive.mount( '/content/drive' )

Mounted at /content/drive


In [ ]:
# 필요한 library 설치

import pandas as pd
import numpy as np
import os
import glob
import random
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
# 데이터 불러오기
data = pd.read_csv( '/content/drive/MyDrive/공모전/B.I. 공모전/B.I Contest/데이터/최종 데이터/최종/dataset.csv')

In [ ]:
data

,Unnamed: 0,기온,강수량,습도,일조,일사,적설,환율,GDP,공휴일,주말,평일,electric,태양광 발전량(MWh),현재수요(MW),현재수요+태양광
0,2013-01-01 00:00:00,-8.0748,0.0,75.2775,0.0,0.0,0.1576,1063.64,131971.70,1.0,0.0,0.0,852.70135,0.0000,61601.19,61601.1900
1,2013-01-01 00:05:00,-8.0859,0.0,75.3719,0.0,0.0,0.1615,1063.64,131970.25,1.0,0.0,0.0,852.70730,0.0000,61250.49,61250.4900
2,2013-01-01 00:10:00,-8.0972,0.0,75.4624,0.0,0.0,0.1615,1063.64,131968.81,1.0,0.0,0.0,852.71320,0.0000,60927.36,60927.3600
3,2013-01-01 00:15:00,-8.1068,0.0,75.5569,0.0,0.0,0.1615,1063.64,131967.38,1.0,0.0,0.0,852.71910,0.0000,60562.58,60562.5800
4,2013-01-01 00:20:00,-8.1176,0.0,75.6525,0.0,0.0,0.1615,1063.64,131965.92,1.0,0.0,0.0,852.72500,0.0000,60521.87,60521.8700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1077403,2023-03-30 23:35:00,10.2794,0.0,49.9997,0.0,0.0,0.0000,1296.55,187184.90,0.0,0.0,1.0,422371.16000,0.1799,62439.54,62439.7199
1077404,2023-03-30 23:40:00,10.2100,0.0,50.1872,0.0,0.0,0.0000,1296.55,187187.17,0.0,0.0,1.0,422373.50000,0.1799,62162.58,62162.7599
1077405,2023-03-30 23:45:00,10.1441,0.0,50.3786,0.0,0.0,0.0000,1296.55,187189.45,0.0,0.0,1.0,422375.88000,0.1799,61694.35,61694.5299
1077406,2023-03-30 23:50:00,10.0753,0.0,50.5654,0.0,0.0,0.0000,1296.55,187191.72,0.0,0.0,1.0,422378.25000,0.1799,61152.42,61152.5999


In [ ]:
data.describe()

,기온,강수량,습도,일조,일사,적설,환율,GDP,공휴일,주말,평일,electric,태양광 발전량(MWh),현재수요(MW),현재수요+태양광
count,1.077408e+06,1.077408e+06,1.077408e+06,1.077408e+06,1.077408e+06,1.077408e+06,1.077408e+06,1.077408e+06,1.077408e+06,1.077408e+06,1.077408e+06,1.077408e+06,1.077408e+06,1.077408e+06,1.077408e+06
mean,1.290485e+01,1.125208e-02,6.734679e+01,2.196421e-02,2.080153e-02,5.591621e-03,1.148278e+03,1.529137e+05,4.116546e-02,2.480620e-01,7.107725e-01,8.309290e+04,9.238984e-01,6.248544e+04,6.248637e+04
std,1.022770e+01,4.701394e-02,1.801741e+01,2.990056e-02,3.059215e-02,2.532604e-02,7.337750e+01,1.830206e+04,1.986729e-01,4.318882e-01,4.534039e-01,1.083185e+05,1.629477e+00,8.482629e+03,8.483034e+03
min,-1.655010e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.009250e+03,1.130031e+05,0.000000e+00,0.000000e+00,0.000000e+00,8.527014e+02,0.000000e+00,3.806520e+04,3.806551e+04
25%,4.379300e+00,0.000000e+00,5.437917e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.103340e+03,1.385376e+05,0.000000e+00,0.000000e+00,0.000000e+00,4.464075e+03,8.000000e-04,5.617210e+04,5.617241e+04
50%,1.361425e+01,0.000000e+00,6.946090e+01,0.000000e+00,5.000000e-04,0.000000e+00,1.134190e+03,1.549466e+05,0.000000e+00,0.000000e+00,1.000000e+00,2.909205e+04,1.346000e-01,6.194807e+04,6.194880e+04
75%,2.167253e+01,3.000000e-04,8.249812e+01,4.710000e-02,3.660000e-02,0.000000e+00,1.185190e+03,1.650257e+05,0.000000e+00,0.000000e+00,1.000000e+00,1.251464e+05,1.132000e+00,6.777070e+04,6.777181e+04
max,3.621520e+01,1.146700e+00,9.840260e+01,8.100000e-02,1.722000e-01,3.945000e-01,1.444630e+03,1.998404e+05,1.000000e+00,1.000000e+00,1.000000e+00,4.223806e+05,1.363810e+01,9.485287e+04,9.485610e+04


In [ ]:
data_2y = pd.DataFrame(data = data['현재수요(MW)'].values, columns = ['현재수요'])

In [ ]:
data_2y

,현재수요
0,61601.19
1,61250.49
2,60927.36
3,60562.58
4,60521.87
...,...
1077403,62439.54
1077404,62162.58
1077405,61694.35
1077406,61152.42


# Denoising Method

1. Wavelet method

2. DAE

In [ ]:
# Wavelet Method

import pywt

def WT(df, col, wavelet='db5', thresh=0.63):
    signal = df[col].values
    thresh = thresh*np.nanmax(signal)
    coeff = pywt.wavedec(signal, wavelet, mode="per" )
    coeff[1:] = (pywt.threshold(i, value=thresh, mode="soft" ) for i in coeff[1:])
    reconstructed_signal = pywt.waverec(coeff, wavelet, mode="per" )
    return reconstructed_signal

data_2y['WT(db5)'] = WT(data_2y, '현재수요(MW)', wavelet = 'db5', thresh = 0.63)
print(data_2y)

In [ ]:
# MAPE 확인

def MAPE(y_test, y_pred):
	return np.mean(np.abs((y_test - y_pred) / y_test)) * 100


In [ ]:
# DAE method

# Min-Max transformation

scaler = MinMaxScaler()

scaled_data_2y = scaler.fit_transform(data_2y)

In [ ]:
scaled_data_2y

array([[0.41445599],
       [0.40828035],
       [0.40259021],
       ...,
       [0.41609649],
       [0.40655339],
       [0.40138942]])

In [ ]:
# Dataset 생성

class CustomDataset(Dataset):

    def __init__(self, input_df):
        self.input_df = input_df

        self.x_data = []
        self.y_data = []

        print("Data_preprocessing...")

        for i in range(len(self.input_df)):

            x = input_df[i]

            # numpy, array, list 인 경우에는 torch.Tensor 써주지만
            # 그냥 단일값인 경우 torch.tensor 사용인 듯
            # 이거 list로 저장하고 2Dimension으로 바꿔줘야할듯

            self.x_data.append(x)
            self.y_data.append(x)

        print("preprocessing done!")

    def __getitem__(self, index):

        x = self.x_data[index]
        y = self.y_data[index]

        return x, y

    def __len__(self):
        return len(self.x_data)

In [ ]:
Dataset = CustomDataset(scaled_data_2y)
Dataloader = DataLoader(Dataset, batch_size = 32, shuffle = False)

Data_preprocessing...
preprocessing done!


In [ ]:
# check the Dataset

Dataset[0]

(array([0.41445599]), array([0.41445599]))

In [ ]:
class TimeDistributed(nn.Module):
    def __init__(self, module):
        super(TimeDistributed, self).__init__()
        self.module = module

    def forward(self, x):
        if len(x.size()) <= 2:
            return self.module(x)
        x_reshape = x.contiguous().view(-1, x.size(-1))
        y = self.module(x_reshape)
        if len(x.size()) == 3:
            y = y.contiguous().view(x.size(0), -1, y.size(-1))
        return y

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.LSTM(
                               input_size = 1,
                               hidden_size = 16,
                               dropout = 0.25,
                               num_layers = 2,
                               bias = True,
                               batch_first = True,
                               bidirectional = True,
                               )
        self.decoder = nn.LSTM(
                               input_size = 32,
                               hidden_size = 16,
                               dropout = 0.25,
                               num_layers = 2,
                               bias = True,
                               batch_first = True,
                               bidirectional = True,
                               )
        self.fc = TimeDistributed(nn.Linear(32, 1))

    def forward(self, x):
        h0, (h_n, c_n) = self.encoder(x)
        h0, (h_n, c_n) = self.decoder(h0[-1:,:].repeat(1,32,1))
        out = self.fc(h0)
        return out

In [ ]:
# device 설정

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
# Seed 고정 함수

def seed_everything(seed: int = 42):

    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(38)

In [ ]:
Denoising_Auto_encoder = Autoencoder().to(device)
optimizer = torch.optim.Adam(Denoising_Auto_encoder.parameters(), lr = 0.005)
criterion = nn.L1Loss().to(device)

# autoencoder train 함수

def train(autoencoder, train_loader):

    autoencoder.train()
    avg_loss = 0

    for X, Y in tqdm(iter(Dataloader)):

        X = X.reshape(-1, 1)
        X = X.to(torch.float32).to(device)

        Y = Y.reshape(-1, 1)
        Y = Y.to(torch.float32).to(device)

        # X = scaler.fit_transform(X)
        # Y = scaler.fit_transform(Y)

        prediction = Denoising_Auto_encoder(X)
        prediction = torch.squeeze(prediction, 0)

        loss = criterion(prediction, Y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_loss += loss.item()

    return avg_loss / len(train_loader)

In [ ]:
# Training / loss 확인

for epoch in range(1, 10):

    loss = train(Denoising_Auto_encoder, Dataloader)
    print("[Epoch {}] loss:{}".format(epoch, loss))


  0%|          | 0/33669 [00:00<?, ?it/s]

[Epoch 1] loss:0.045823540290512076


  0%|          | 0/33669 [00:00<?, ?it/s]

[Epoch 2] loss:0.020916306933974473


  0%|          | 0/33669 [00:00<?, ?it/s]

...

[Epoch 7] loss:0.013848118354317197


  0%|          | 0/33669 [00:00<?, ?it/s]

[Epoch 8] loss:0.0130723022270979


  0%|          | 0/33669 [00:00<?, ?it/s]

[Epoch 9] loss:0.012442239156749286


In [ ]:
# Test
# Input은 Training에 사용했던 것과 동일하게 사용

def DAE_test(testloader):

    pred_result = []

    for X, Y in testloader:
            X = X.reshape(-1, 1)
            X = X.to(torch.float32).to(device)

            pred = Denoising_Auto_encoder(X)

            pred = torch.squeeze(pred, 0)
            pred = torch.squeeze(pred, 1)
            pred = pred.tolist()
            for i in pred:
                pred_result.append(i)

    return pred_result

test_result = DAE_test(Dataloader)

In [ ]:
# 정규화 역변환 과정

test_result = np.array(test_result)
test_result = test_result.reshape(-1, 1)
real_test_result = scaler.inverse_transform(test_result)

In [ ]:
real_test_result

array([[61554.68722827],
       [61166.94721458],
       [61240.11832093],
       ...,
       [61695.9437656 ],
       [61165.86069092],
       [61490.27262254]])

In [ ]:
data_2y['DAE'] = real_test_result

In [ ]:
MAPE(data_2y['현재수요'], data_2y['DAE'])

1.5459053172242014

In [ ]:
data_2y.to_csv("DAE_result_not_sunpower.csv")

# Stationary Test

1. ADF Test

2. KPSS Test

In [ ]:
import seaborn as sns
from statsmodels.tsa.stattools import adfuller, kpss

In [ ]:
# ADF Test


def adf_test(timeseries, pvalue = .05, regression_option = 'ct'):
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC', regression = regression_option)
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)
    if dfoutput[1] < pvalue:
       print(f"정상시계열이 아니라는 귀무가설을 {pvalue*100}%의 유의수준으로 기각할 수 있으므로 해당 데이터는 정상성이 보장됩니다.")
    else:
       print(f"정상시계열이 아니라는 귀무가설을 {pvalue*100}%의 유의수준으로 기각할 수 없으므로 해당 데이터는 정상성을 보장하지 못합니다.")

In [ ]:
adf_test(data_2y['DAE'])

In [ ]:
# KPSS Test

def kpss_test(timeseries, pvalue = .05, regression_option = 'ct'):
    print ('Results of KPSS Test:')
    kpsstest = kpss(timeseries, regression= regression_option)
    kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic','p-value','Lags Used'])
    for key,value in kpsstest[3].items():
        kpss_output['Critical Value (%s)'%key] = value
    print (kpss_output)
    if kpss_output[1] < pvalue:
        print(f"정상시계열이 맞다는 귀무가설을 {pvalue*100}%의 유의수준으로 기각할 수 있으므로 해당 데이터는 정상성을 보장하지 못합니다.")
    else:
        print(f"정상시계열이 맞다는 귀무가설을 {pvalue*100}%의 유의수준으로 기각할 수 없으므로 해당 데이터는 정상성이 보장됩니다.")

In [ ]:
kpss_test(data_2y['WT(db5)'])